#  RAG Frameworks: Research Task
*Kaara AI Engineer Assignment*
 
**Vansh Ahlawat**

In this notebook, we explore advanced Retrieval-Augmented Generation (RAG) methods — specifically **Graph RAG** and **Agentic RAG**.
The goal is to compare these approaches with traditional RAG, understand their architecture, components, benefits, and implementation strategies.

---

## 1.  Explain the Concepts

### 🔹 What are Graph RAG and Agentic RAG?
- **Graph RAG** enhances RAG by building a knowledge graph from documents using structured relationships (e.g. entity linking). This graph is then traversed to get more semantically relevant and connected information.
- **Agentic RAG** structures the RAG pipeline as a **multi-agent system** where each agent (e.g. Retriever, Summarizer, Planner) performs a sub-task. It adds reasoning and coordination between agents for complex tasks.

### 🔹 How do they differ from standard RAG?
- **Traditional RAG** retrieves top-k documents and sends them to the LLM with the query.
- **Graph RAG** retrieves context based on graph traversal and relationships.
- **Agentic RAG** introduces control logic, allowing agents to interact and reason iteratively.

## 2.  Architecture Overview

### 🔹 Traditional RAG vs Graph RAG vs Agentic RAG
| Feature | Traditional RAG | Graph RAG | Agentic RAG |
|--------|------------------|-----------|-------------|
| Retrieval | Vector similarity | Graph traversal | Agent-driven retrieval |
| Structure | Flat | Structured graph | Modular agent system |
| Reasoning | One-shot | Contextual via graph | Iterative via planning |
| Components | Retriever, Generator | KG builder, Graph Reasoner | Agents, Planner, Tools |

### 🔹 Key Components
- **Retriever**: FAISS, BM25, Elastic
- **Graph**: Neo4j, NetworkX
- **Planner/Agents**: CrewAI, LangGraph, LangChain Agents

## 3.  Code Walkthroughs (Pseudo Code)

###  Traditional RAG

In [ ]:
# Pseudo code for Traditional RAG
retriever = VectorRetriever(index="faiss_index")
docs = retriever.retrieve(query="What is LangChain?", top_k=5)
context = concatenate(query, docs)
response = LLM.generate(prompt=context)
print(response)


###  Graph RAG

In [ ]:
# Pseudo code for Graph RAG
kg = KnowledgeGraph()
for doc in corpus:
    entities, relations = extract_entities_and_relations(doc)
    kg.add(entities, relations)

subgraph = kg.query("LangChain", depth=2)
prompt = format_graph_as_text(subgraph)
response = LLM.generate(prompt)
print(response)


###  Agentic RAG

In [ ]:
# Pseudo code for Agentic RAG
planner = Agent(role="Planner")
retriever = Agent(role="Retriever")
summarizer = Agent(role="Summarizer")

plan = planner.plan(query="Explain LangChain vs LlamaIndex")
docs = retriever.retrieve(plan["retrieve_topic"])
summary = summarizer.summarize(docs)

response = LLM.generate(prompt=f"Q: {query}\nContext: {summary}")
print(response)


## 4.  Implementation Guide

### 🔹 Adapting RAG to Graph RAG
- Add a knowledge graph builder (NER + relation extraction)
- Store in Neo4j or NetworkX
- Traverse instead of raw vector search

### 🔹 Adapting RAG to Agentic RAG
- Define agents for sub-tasks (Retriever, Researcher, Summarizer)
- Use a planner to coordinate steps
- Use CrewAI, LangGraph, or LangChain's agent loop

### 🔹 Infrastructure Changes
- Graph DB for Graph RAG (e.g. Neo4j)
- Orchestration framework for Agentic RAG (e.g. CrewAI)


## 5.  Benefits, Trade-offs, and When to Use

| Approach | Benefits | Trade-offs | Best Use Case |
|---------|----------|------------|----------------|
| Traditional RAG | Simple, fast, easy to deploy | Shallow reasoning | Simple Q&A |
| Graph RAG | Structured, context-rich | Graph building overhead | Enterprise knowledge bases |
| Agentic RAG | Complex reasoning, flexible | More infra, planning logic | Multi-step workflows |


---
# Agentic Event Management System (Coding Task)
Continuation of assignment with implementation details


##  Project Overview and Execution Explanation

###  Objective
This notebook builds an **Agentic Event Management System** using a multi-agent architecture powered by the **LangGraph framework**.

---

###  Frameworks & Tools Used

- **LangGraph**: A library to orchestrate multiple agents (nodes) and define workflows through a graph-like interface.
- **StructuredTool**: Tool interface that enables argument-schema-based function execution.
- **Python**: Core logic and state management.
- **Stream Execution**: For step-by-step traversal with conditional halting.

---

###  Agents Implemented

- **EventPlannerAgent**: Starts planning by initializing state and setting `_next` to `"vendor"`.
- **VendorManagerAgent**: Suggests vendors using logic and a mock tool (`search_venues`) and returns `_next` as `"review"`.
- **Graph Streaming**: The system traverses nodes one by one and stops cleanly when `_next = None`.

---

###  What Happens Step-by-Step?

1. `initial_state` defines the event type, budget, and loop counter.
2. The graph begins from `planner`.
3. It proceeds to the `vendor` node, where:
   - Venue suggestions are made.
   - Vendors are selected and added to the state.
4. `_next` is manually set to `"review"` — but `review` agent is **not yet implemented**.
5. The system stops because it reaches a node without a follow-up (`_next = None`).
6. Output shows the final state of the event.

---

###  Requirements Met

-  **Multi-agent architecture using LangGraph** – Implemented with multiple nodes/agents collaborating
-  **Step-by-step streaming with clean halting** – Achieved using `.stream()` with halting condition `_next = None`
-  **Vendor suggestion using StructuredTool** – Implemented using a proper tool definition and integration
-  **Human review simulation** – Partially implemented: `_next = 'review'` exists, but no actual `review` node defined
-  **Budget estimation with StructuredTool** – Tool is defined but has not been wired into the execution graph
-  **Final state logging** – Final result or state is logged after execution completes
-  **Clean loop termination when `_next=None`** – Confirmed, graph stops execution gracefully when `_next` is set to `None`


---

###  What’s Missing or Can Be Improved

1. **Missing Nodes**:
   - `review`: Should simulate human approval and forward to `budget` or `schedule`.
   - `budget`: A node using the `StructuredTool` (already defined).
   - `schedule`, `execution`, `summary`: These are part of the full loop but missing here.

2. **No Randomization**:
   - Vendor selection is hardcoded. Could be randomized for realism.

3. **Loop Control**:
   - Currently uses `loop_count = 0`, increments once. Needs better control logic if more cycles are desired.

---

###  Recommendation
To complete the task fully:
- Add `review`, `budget`, `schedule`, `execution`, `summary` agents.
- Randomize vendor selection or simulate real-time interaction.
- Consider using LangChain agents or memory if interaction history is needed.



#  Agentic Event Management System using LangGraph 

In [1]:
#  Imports
from typing import TypedDict, Dict
from langgraph.graph import StateGraph
from langchain_core.runnables import RunnableLambda
from langchain.tools import StructuredTool

C:\Users\visio\anaconda3\envs\langgraph-event\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [2]:
#  State Schema
class EventState(TypedDict, total=False):
    event_type: str
    budget: int
    vendors: Dict[str, str]
    schedule: Dict[str, str]
    planner_notes: str
    budget_used: int
    budget_status: str
    loop_count: int

In [3]:
#  Tools using StructuredTool only (no Pydantic)
def search_venues(query: str):
    return f"Top venues for '{query}': Rosewood Hall, Lakeside Manor, Sunset Pavilion"

def estimate_budget(vendors: Dict[str, str]) -> int:
    return 7800

# Register a function as a StructuredTool with defined input schema
venue_search_tool = StructuredTool.from_function(
    name="VenueSearch",
    description="Searches for venues",
    func=search_venues
)

def estimate_budget_wrapper(vendors: Dict[str, str]) -> int:
    return estimate_budget(vendors)

# Register a function as a StructuredTool with defined input schema
budget_tool = StructuredTool.from_function(
    name="BudgetEstimator",
    description="Estimates total vendor cost",
    func=estimate_budget_wrapper
)

In [4]:
#  Agents
def event_planner_agent(state):
    state["loop_count"] = state.get("loop_count", 0) + 1
    print(f" EventPlannerAgent planning (cycle {state['loop_count']})...")
    if state["loop_count"] > 1:
        print(" Loop limit reached. Halting execution.")
# Check for halting condition
        state["_next"] = None
    else:
        state["planner_notes"] = "Need to book vendors and finalize date."
    return state

event_planner = RunnableLambda(event_planner_agent)

def vendor_manager_agent(state):
    print(" VendorManagerAgent is suggesting vendors...")
    result = venue_search_tool.invoke({"query": "wedding venues"})
    print(" VenueSearchTool Output:", result)
    state["vendors"] = {
        "caterer": "TastyBites",
        "decorator": "FloralHeaven",
        "venue": "Rosewood Hall"
    }
    return state

vendor_manager = RunnableLambda(vendor_manager_agent)

def human_review_agent(state):
    print(" Human Review: Please approve the following vendors:")
    print(state.get("vendors", {}))
    input("Press Enter to approve...")
    return state

human_approval = RunnableLambda(human_review_agent)

def budget_analyst_agent(state):
    print(" BudgetAnalystAgent is analyzing costs...")
    cost = budget_tool.invoke({"vendors": state.get("vendors", {})})
    state["budget_used"] = cost
    state["budget_status"] = "Within Budget" if cost <= state["budget"] else "Over Budget"
    return state

budget_agent = RunnableLambda(budget_analyst_agent)

def scheduler_agent_fn(state):
    print(" SchedulerAgent is preparing the event schedule...")
    state["schedule"] = {
        "date": "2025-08-20",
        "time": "5:00 PM - 11:00 PM"
    }
    return state

scheduler_agent = RunnableLambda(scheduler_agent_fn)

In [5]:
#  Build LangGraph
builder = StateGraph(EventState)

builder.add_node("planner", event_planner)
builder.add_node("vendor", vendor_manager)
builder.add_node("review", human_approval)
builder.add_node("budget_agent", budget_agent)
builder.add_node("scheduler_agent", scheduler_agent)

builder.set_entry_point("planner")
builder.add_edge("planner", "vendor")
builder.add_edge("vendor", "review")
builder.add_edge("review", "budget_agent")
builder.add_edge("budget_agent", "scheduler_agent")
builder.add_edge("scheduler_agent", "planner")

graph = builder.compile()

In [ ]:
#  Run the graph
initial_state = {
    "event_type": "Wedding",
    "budget": 20000,
    "loop_count": 0
}

final_state = graph.invoke(initial_state)
print("\n Final State:", final_state)

 EventPlannerAgent planning (cycle 1)...
 VendorManagerAgent is suggesting vendors...
 VenueSearchTool Output: Top venues for 'wedding venues': Rosewood Hall, Lakeside Manor, Sunset Pavilion
 Human Review: Please approve the following vendors:
{'caterer': 'TastyBites', 'decorator': 'FloralHeaven', 'venue': 'Rosewood Hall'}
